# Lesson 4: Persistence and Streaming

In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=2)

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [5]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [6]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [7]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4o")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [8]:
messages = [HumanMessage(content="What is the weather in sf?")]

In [9]:
thread = {"configurable": {"thread_id": "1"}}

In [10]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LaH9ju7P3TwSoHM6RSJH4bz8', 'function': {'arguments': '{"query":"current weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 151, 'total_tokens': 173}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-aa0718ea-dc4e-4f07-9675-47a80f8d2e15-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_LaH9ju7P3TwSoHM6RSJH4bz8'}])]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_LaH9ju7P3TwSoHM6RSJH4bz8'}
Back to the model!
[ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America

In [13]:
from IPython.display import Markdown
Markdown(v['messages'][0].content)

The current weather in San Francisco is as follows:
- **Temperature:** 13.9°C (57.0°F)
- **Condition:** Partly cloudy
- **Wind:** 20.6 mph (33.1 kph) from the west
- **Humidity:** 89%
- **Visibility:** 14 km (8 miles)
- **Pressure:** 1013 mb (29.9 in)
- **Feels Like:** 12.3°C (54.2°F)

Additionally, there are no significant precipitation events, and the weather is generally mild with some cloud cover.

In [14]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nZGxY49PO9Ehg2iGE5KWSD9Z', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 2403, 'total_tokens': 2425}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7186b32f-1b73-465d-b7e7-f0a6651a203c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_nZGxY49PO9Ehg2iGE5KWSD9Z'}])]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_nZGxY49PO9Ehg2iGE5KWSD9Z'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'Uni

In [15]:
Markdown(v['messages'][0].content)

The current weather in Los Angeles is as follows:
- **Temperature:** 18.3°C (64.9°F)
- **Condition:** Overcast
- **Wind:** 2.2 mph (3.6 kph) from the north
- **Humidity:** 81%
- **Visibility:** 16 km (9 miles)
- **Pressure:** 1013 mb (29.9 in)
- **Feels Like:** 18.3°C (64.9°F)

The weather is generally overcast with mild temperatures and light winds.

In [16]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Los Angeles is currently warmer than San Francisco. Los Angeles has a temperature of 18.3°C (64.9°F), while San Francisco has a temperature of 13.9°C (57.0°F).', response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 3047, 'total_tokens': 3092}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-7f7c17c6-f841-42c6-b898-7828a6f78118-0')]}


In [17]:
Markdown(v['messages'][0].content)

Los Angeles is currently warmer than San Francisco. Los Angeles has a temperature of 18.3°C (64.9°F), while San Francisco has a temperature of 13.9°C (57.0°F).

In [18]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Could you please clarify what you are comparing to determine which one is warmer? Are you comparing two specific locations, materials, clothing items, or something else?', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 149, 'total_tokens': 181}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-125247b2-f688-41e8-a6ee-653e3cb21ff6-0')]}


In [19]:
Markdown(v['messages'][0].content)

Could you please clarify what you are comparing to determine which one is warmer? Are you comparing two specific locations, materials, clothing items, or something else?

## Streaming tokens

In [22]:
# %pip install aiosqlite

In [26]:
from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

memory = AsyncSqliteSaver.from_conn_string(":memory:")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [27]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_GC86BjZN3DGlp02vyMaPm2Fq'}
Back to the model!
